In [30]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#  CRUD Python module file name and class name
from animal_shelter_CRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = 'aacuser'
password = 'ACC_USER_PASS'
host = 'nv-desktop-services.apporto.com'
port = 30258
database = 'AAC'
collection = 'animals'

# Connect to database via CRUD Module
db = AnimalShelter(username, password, host, port, database, collection)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'Grazioso-Salvare-Logo.png' # replaced with Grazioso Salvare’s logo
try: 
    encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()
    img_html = html.A(
    html.Img(src =f'data:image/png;base64,{encoded_image}', style ={'height': '300px'}),
        href="https://www.snhu.edu",
        target = "_blank"
    )
except FileNotFoundError:
    img_html = html.P("Logo not found.")

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Center("Ericka Resendez Project Two Dashboard"),
    html.Hr(),
    html.Center(img_html), #hyperlinked logo as per instructions
    html.Hr(),
    # interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    html.Div(
        dcc.RadioItems(
            id = 'filter_type',
            options = [
            {'label': 'Water Rescue', 'value': 'water-btn'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'wilderness-btn'}, 
            {'label': 'Disaster or Individual Tracking', 'value' : 'disaster-btn'},
            {'label': 'Reset', 'value' : 'reset'}
        ], 
        value = 'reset', 
        inline = True        
        )
    ),
    html.Hr(),
    dash_table.DataTable(
                         id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         style_table={'overflowX': 'auto'},
                         style_data = {'whiteSpace': 'normal', 'height': 'auto'}, 
                         style_header = {'backgroundColor': 'lightgrey', 'fontWeight': 'bold'},
                         editable= False, # copied code from module six assignment
                         page_action= "native", 
                         page_current= 0,
                         page_size= 10, #limiting page to 10 rows
                         filter_action= "native", 
                         sort_action= "native",
                         sort_mode= "multi", 
                         row_selectable= 'single',
                         row_deletable = False,
                         selected_rows= [],
                         selected_columns=[]
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ]),
    html.Div(id="query-out", style={'whiteSpace': 'pre-line'}),
    # inserting a unique identifier code here.
    html.H1("Ericka's Client Server Authentication"),
])

#############################################
# Interaction Between Components / Controller
#############################################

    
@app.callback([Output('datatable-id','data'),
              Output('datatable-id', 'columns')],
              [Input('filter_type', 'value')])

def update_dashboard(filter_type):
    
    filters = { #water rescue filter
        "water-btn": {
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, 
            "sex_upon_outcome": "Intact Female", 
            "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte":156.0}},
        
        # mountain or wilderness rescue filter
        "wilderness-btn": {
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}, 
            "sex_upon_outcome": "Intact Male", 
            "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte":156.0}},
        
        # disaster or individual tracking filter
        "disaster-btn":{
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}, 
            "sex_upon_outcome": "Intact Male", 
            "age_upon_outcome_in_weeks": {"$gte": 20.0, "$lte":300.0}}
        
    }
    
    query = filters.get(filter_type, {})
            
    df_filtered = pd.DataFrame.from_records(db.read(query))
    
    if df_filtered.empty:
        return [], []
        
    df_filtered.drop(columns = ['_id'], inplace = True, errors = "ignore")
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df_filtered.columns]
        
    return df_filtered.to_dict('records'), columns


@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])

def update_graphs(viewData):
    if viewData is None or len(viewData) ==0: 
        return [dcc.Graph(figure=px.pie(title= "No data available"))]
    
    dff = pd.DataFrame.from_dict(viewData)
    
    if 'breed' not in dff.columns or dff.empty: 
        return [dcc.Graph(figure=px.pie(title="No valid data to display"))]
    
    breed_counts = dff['breed'].value_counts().reset_index()
    breed_counts.columns = ['breed', 'count']
    
    fig = px.pie(data_frame = breed_counts, 
                    values = 'count', 
                    names = 'breed', 
                    color_discrete_sequence = px.colors.sequential.RdBu, 
                    title = "Breed Distribution",
                    width = 800, height= 500
                )
    
    return [dcc.Graph(figure=fig)]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:19925/
